# Mistral Benchmarking & Prompt Completion evaluation using SageMaker Jumpstart

---
Let's do some text generation, benchmarking and evaluation for Mistral models on sagemaker jumpstart. You can easily deploy the model on jumpstart.

---

## Setup
***

In [2]:
model_id = "huggingface-llm-mistral-7b-instruct"

In [3]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id)
predictor = model.deploy()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


Using model 'huggingface-llm-mistral-7b-instruct' with wildcard version identifier '*'. You can pin to version '2.0.0' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


--------!

### Supported parameters

***
This model supports many parameters while performing inference. They include:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **max_new_tokens:** Model generates text until the output length (excluding the input context length) reaches `max_new_tokens`. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **return_full_text:** If True, input text will be part of the output generated text. If specified, it must be boolean. The default value for it is False.
* **stop**: If specified, it must a list of strings. Text generation stops if any one of the specified strings is generated.

We may specify any subset of the parameters mentioned above while invoking an endpoint. Next, we show an example of how to invoke endpoint with these arguments
***

## Instruction prompts
***
The examples in this section demonstrate queries to the Mistral 7B Instruct model. This involves special token formatting within the prompt input. The base, pre-trained Mistral model is not fine-tuned to perform this instruction task -- please use the prompts in the next section instead.
***

In [4]:
from typing import Dict, List


def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for user, answer in zip(instructions[::2], instructions[1::2]):
        prompt.extend(["<s>", "[INST] ", (user["content"]).strip(), " [/INST] ", (answer["content"]).strip(), "</s>"])
    prompt.extend(["<s>", "[INST] ", (instructions[-1]["content"]).strip(), " [/INST] "])
    return "".join(prompt)


def print_prompt_and_response(prompt: str, response: str) -> None:
    bold, unbold = '\033[1m', '\033[0m'
    print(f"{bold}> Input{unbold}\n{prompt}\n\n{bold}> Output{unbold}\n{response[0]['generated_text']}\n")

In [5]:
instructions = [{"role": "user", "content": "what is hindi?"}]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256, "do_sample": True}
}
response = predictor.predict(payload)
print_prompt_and_response(prompt, response)

> Input
<s>[INST] what is the recipe of mayonnaise? [/INST] 

> Output
1. 1/2 cup egg yolks
2. 1 tablespoon Dijon mustard
3. 2 tablespoons lemon juice
4. 1/2 teaspoon salt
5. 3/4 cup vegetable oil or canola oil
6. Equipment: whisk, cutting board, measure, mixing bowl.



In [6]:
instructions = [
    {"role": "user", "content": "I am going to Paris, what should I see?"},
    {
        "role": "assistant",
        "content": """\
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
    },
    {"role": "user", "content": "What is so great about #1?"},
]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256, "do_sample": True}
}
response = predictor.predict(payload)
print_prompt_and_response(prompt, response)

> Input
<s>[INST] I am going to Paris, what should I see? [/INST] Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.</s><s>[INST] What is so great about #1? [/INST] 

> Output
1. The Eiffel Tower:

## Pre-trained model prompts
***
The examples in this section demonstrate how to perform text generation on the base, pre-trained Mistral model. If you have deployed the instruction-tuned model, please use prompt formatting in the previous section instead.
***

In [9]:
prompt = "Write a program to compute factorial in python:"
payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 200,
    },
}
response = predictor.predict(payload)
print_prompt_and_response(prompt, response)

> Input
Write a program to compute factorial in python:

> Output


```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

n = int(input("Enter a number: "))
result = factorial(n)
print("Factorial of", n, "is", result)
```

In this program, we define a function named `factorial` that takes an integer `n` as an argument. Inside the function, we use recursion to compute the factorial of the given number. If the value of `n` is `0`, then the function returns `1`. Otherwise, it returns the product of `n` and the factorial of `n-1`.

We then take an input from the user to compute the factorial of a given number. We call the `factorial` function with the input value and store the result in a



In [10]:
prompt = "Building a website can be done in 10 simple steps:"
payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 110,
        "no_repeat_ngram_size": 3,
    },
}
response = predictor.predict(payload)
print_prompt_and_response(prompt, response)

> Input
Building a website can be done in 10 simple steps:

> Output


1. Choose a domain name: This is the first step in creating a website. Choose a name that represents your brand or business.

2. Choose a hosting provider: A hosting provider is a company that provides the server space where your website will be stored.

3. Choose a website builder: A website builder is a tool that allows you to create a website without any coding knowledge.

4. Choose a website template: A website template is a pre-designed layout that you can customize to fit your



## Now - Let's deploy Mistral 7b itself and test it on some questions

In [16]:
model_id_noinstruct = "huggingface-llm-mistral-7b"

In [19]:
from sagemaker.jumpstart.model import JumpStartModel

model_new = JumpStartModel(model_id=model_id)
predictor_new = model_new.deploy()

---------!

In [20]:
prompt = "Building a website can be done in 10 simple steps:"
payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 110,
        "no_repeat_ngram_size": 3,
    },
}
response = predictor_new.predict(payload)
print_prompt_and_response(prompt, response)

> Input
Building a website can be done in 10 simple steps:

> Output


1. Choose a domain name: This is the first step in creating a website. Choose a name that represents your brand or business.

2. Choose a hosting provider: A hosting provider is a company that provides the server space where your website will be stored.

3. Choose a website builder: A website builder is a tool that allows you to create a website without any coding knowledge.

4. Choose a website template: A website template is a pre-designed layout that you can customize to fit your



## Now, let's load the aisquared/dais-question-answers dataset to test for some latency :)

In [23]:
%pip install --upgrade sagemaker datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 195.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 246.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 241.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 222.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 230.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 282.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 321.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 294.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 203.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 314.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.8/308.8 kB 165.4 MB/s eta 0:00:00
DEPRECATION: pyodbc 4.0.0-unsupported has 

In [26]:
from datasets import load_dataset

dais = load_dataset("aisquared/dais-question-answers", split="train")

# We split the dataset into two where test data is used to evaluate at the end.
train_and_test_dataset = dais.train_test_split(test_size=0.1)

# Dumping the training data to a local file to be used for training.
train_and_test_dataset["train"].to_json("train.jsonl")

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

840825

In [28]:
prompt_inference = ("""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{context}""")

In [30]:
import pandas as pd
import time
from IPython.display import HTML, display

# Assuming test_dataset is already defined
test_dataset = train_and_test_dataset["test"]

def predict_and_print(datapoint, predictor):
    payload = {
        "inputs": datapoint["question"],
        "parameters": {"max_new_tokens": 100},
    }

    start_time = time.time()
    response = predictor.predict(payload)
    end_time = time.time()

    latency = end_time - start_time
    response_text = response["generated_text"] if isinstance(response, dict) else response

    return {
        "Question": datapoint["question"],
        "Ground Truth Answer": datapoint["answer"],
        "Model Prediction": response_text,
        "Inference Latency (seconds)": latency
    }

results_new = []
results_old = []

try:
    for i, datapoint in enumerate(test_dataset.select(range(10))):
        result_new = predict_and_print(datapoint, predictor_new)
        result_old = predict_and_print(datapoint, predictor)
        results_new.append(result_new)
        results_old.append(result_old)

    df_new = pd.DataFrame(results_new)
    df_old = pd.DataFrame(results_old)

    print("Results from predictor_new:")
    display(HTML(df_new.to_html()))

    print("Results from predictor:")
    display(HTML(df_old.to_html()))

except Exception as e:
    print(e)



Results from predictor_new:


,Question,Ground Truth Answer,Model Prediction,Inference Latency (seconds)
0,What is the Centro de demonstração de produtos e parcerias da Databricks and what services does it offer?,"The Centro de demonstração de produtos e parcerias da Databricks is a platform that offers demonstrations of solution accelerators. It also offers services such as documentation, training and certification, resources, online community, alliance with universities, events, and partnership programs for cloud partners and technology and data partners. Additionally, it provides solutions for partners to connect with validated partner solutions.","[{'generated_text': ' The Centro de demonstração de produtos e parcerias da Databricks is a platform that allows users to test and evaluate Databricks products and services. It offers a range of features and services, including: 1. Data Science Workspace: A cloud-based platform that provides a complete environment for data scientists to develop, test, and deploy machine learning models. 2. Apache Spark: An open-source distributed computing engine that'}]",3.368706
1,What is the purpose of the Data and AI Summit 2023?,"The purpose of the Data and AI Summit 2023 is to bring together experts, researchers, contributors, and professionals from the global data community to understand the potential of Large Language Models (LLMs) and shape the future of their industries with data and AI. Attendees can expect to hear from top speakers in the field, including those from Databricks and across the data and AI community, and learn about building, training, and deploying LLMs, as well as other relevant topics in data and AI.","[{'generated_text': ' The purpose of the Data and AI Summit 2023 is to bring together industry leaders, experts, and professionals to discuss the latest trends, innovations, and challenges in the field of data and artificial intelligence. The summit aims to provide a platform for knowledge sharing, networking, and collaboration, and to inspire attendees to drive the adoption and implementation of data and AI technologies in their organizations.'}]",2.803292
2,What AWS competencies did Databricks showcase at AWS re:Invent?,"Databricks showcased AWS competencies in data engineering, data warehousing, data streaming, and machine learning at AWS re:Invent.","[{'generated_text': ' At AWS re:Invent, Databricks showcased several AWS competencies, including: 1. Big Data: Databricks is a leading big data platform that leverages AWS services such as Amazon S3, Amazon EMR, and Amazon Redshift to process and analyze large volumes of data. 2. Machine Learning: Databricks provides a comprehensive machine learning platform that integrates with AWS services such as Amazon SageMaker, Amazon K'}]",3.293204
3,What is the date and time of the upcoming webinar about transitioning from data warehouse to data lakehouse?,The upcoming webinar about transitioning from data warehouse to data lakehouse is on May 18 at 8 AM PT.,[{'generated_text': ' The upcoming webinar about transitioning from data warehouse to data lakehouse is on [insert date and time here].'}],0.863199
4,What are Databricks Solution Accelerators and how do they help deliver data and AI value faster?,"Databricks Solution Accelerators are tools developed by Databricks that help deliver data and AI value faster. They save time in the discovery, design, development, and testing phases by providing pre-built solutions for common use cases in industries such as Financial Services, Healthcare, Manufacturing, and more. These accelerators allow organizations to quickly implement data and AI solutions, enabling them to achieve their business objectives faster.","[{'generated_text': ' Databricks Solution Accelerators are pre-built, reusable components that help organizations deliver data and AI value faster. These accelerators are designed to address common data and AI challenges and provide a quick and easy way to get started with Databricks. There are several types 

Results from predictor:


,Question,Ground Truth Answer,Model Prediction,Inference Latency (seconds)
0,What is the Centro de demonstração de produtos e parcerias da Databricks and what services does it offer?,"The Centro de demonstração de produtos e parcerias da Databricks is a platform that offers demonstrations of solution accelerators. It also offers services such as documentation, training and certification, resources, online community, alliance with universities, events, and partnership programs for cloud partners and technology and data partners. Additionally, it provides solutions for partners to connect with validated partner solutions.","[{'generated_text': ' The Centro de demonstração de produtos e parcerias da Databricks is a platform that allows users to test and evaluate Databricks products and services. It offers a range of features and services, including: 1. Data Science Workspace: A cloud-based platform that provides a complete environment for data scientists to develop, test, and deploy machine learning models. 2. Apache Spark: An open-source distributed computing engine that'}]",3.315304
1,What is the purpose of the Data and AI Summit 2023?,"The purpose of the Data and AI Summit 2023 is to bring together experts, researchers, contributors, and professionals from the global data community to understand the potential of Large Language Models (LLMs) and shape the future of their industries with data and AI. Attendees can expect to hear from top speakers in the field, including those from Databricks and across the data and AI community, and learn about building, training, and deploying LLMs, as well as other relevant topics in data and AI.","[{'generated_text': ' The purpose of the Data and AI Summit 2023 is to bring together industry leaders, experts, and professionals to discuss the latest trends, innovations, and challenges in the field of data and artificial intelligence. The summit aims to provide a platform for knowledge sharing, networking, and collaboration, and to inspire attendees to drive the adoption and implementation of data and AI technologies in their organizations.'}]",2.876742
2,What AWS competencies did Databricks showcase at AWS re:Invent?,"Databricks showcased AWS competencies in data engineering, data warehousing, data streaming, and machine learning at AWS re:Invent.","[{'generated_text': ' At AWS re:Invent, Databricks showcased several AWS competencies, including: 1. Big Data: Databricks is a leading big data platform that leverages AWS services such as Amazon S3, Amazon EMR, and Amazon Redshift to process and analyze large volumes of data. 2. Machine Learning: Databricks provides a comprehensive machine learning platform that integrates with AWS services such as Amazon SageMaker, Amazon K'}]",3.294858
3,What is the date and time of the upcoming webinar about transitioning from data warehouse to data lakehouse?,The upcoming webinar about transitioning from data warehouse to data lakehouse is on May 18 at 8 AM PT.,[{'generated_text': ' The upcoming webinar about transitioning from data warehouse to data lakehouse is on [insert date and time here].'}],0.864549
4,What are Databricks Solution Accelerators and how do they help deliver data and AI value faster?,"Databricks Solution Accelerators are tools developed by Databricks that help deliver data and AI value faster. They save time in the discovery, design, development, and testing phases by providing pre-built solutions for common use cases in industries such as Financial Services, Healthcare, Manufacturing, and more. These accelerators allow organizations to quickly implement data and AI solutions, enabling them to achieve their business objectives faster.","[{'generated_text': ' Databricks Solution Accelerators are pre-built, reusable components that help organizations deliver data and AI value faster. These accelerators are designed to address common data and AI challenges and provide a quick and easy way to get started with Databricks. There are several types 

## Model evaluation with OS eval frameworks

In [31]:
!pip install uptrain

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.8/228.8 kB 210.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 225.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 193.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 243.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 234.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 205.0 MB/s eta 0:00:00
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attemptin

## Clean up the endpoint

In [15]:
# predictor.delete_model()
# predictor.delete_endpoint()